<a href="https://colab.research.google.com/github/Noha-Magdy/Text-AudioDatasets/blob/main/Librispeech/data_download_and_save.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
# edits to be done in the data before loading it
# edit line 45 remove || 
# edit ;id to be id 
#-----------------------------------------
# use sep | and skip first 11 rows 

pd.read_csv('SPEAKERS.TXT', sep="|", skiprows=11)

,ID,SEX,SUBSET,MINUTES,NAME
0,14,F,train-clean-360,25.03,Kristin LeMoine
1,16,F,train-clean-360,25.11,Alys AtteWater
2,17,M,train-clean-360,25.04,Gord Mackenzie
3,19,F,train-clean-100,25.19,Kara Shallenberg
4,20,F,train-other-500,30.07,Gesine
...,...,...,...,...,...
2479,8975,F,train-clean-100,25.11,Daisy Flaim
2480,9000,M,train-other-500,27.26,Ramon Escamilla
2481,9022,F,train-clean-360,25.17,Claire M
2482,9023,F,train-clean-360,25.19,P. J. Morgan


In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=eeebb74b5692b24b6ac4b43d44004635a69c9891a89a057d2c20e6f00c4f1c23
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
# Downloads librispeech
import wget

def download_librispeech(out, splits):

  def _splits_url(split_name):
    return "https://www.openslr.org/resources/12/"+split_name+".tar.gz"

  for split in splits:
    wget.download(splits_url(split), out=out, bar="bar_thermometer")

In [ ]:
def unzip_librispeech(out, splits):
  

In [ ]:
# splits are:

dev-clean
dev-other
test-clean 
test-other
train-clean-100
train-clean-360
train-other-500
